# 第7章 基于RNN生成文本

## 使用语言模型生成文本

### 使用RNN生成文本的步骤

上一章的模型

![](./img/7-1.png)

问题：语言模型输出了当给定单词 i 时下一个出现的单词的概率分布。那么，它如何生成下一个新单词呢？

- 一选择概率最高的单词，这是一种“确定性的”方法。

- 另一种方法是“概率性地”进行选择。根据概率分布进行选择，这样概率高的单词容易被选到，概率低的单词难以被选到。在这种情况下，被选到的单词（被采样到的单词）每次都不一样。

回到我们的例子中，如图所示，假设（概率性地）选择了单词 say。

![](./img/7-2.png)

概率分布中，say 的概率最高，所以它被采样到的概率也最高。

不过请注意，这里选到 say 并不是必然的（不是确定性的），而是概率性的。

因此，say 以外的其他单词根据出现的概率也可能被采样到。

![](./img/7-3.png)

接下来，采样第 2 个单词。这只需要重复一下刚才的操作。也就是说，
将生成的单词 say 输入语言模型，获得单词的概率分布，然后再根据这个概
率分布采样下一个出现的单词，如图 7-4 所示。

![](./img/7-4.png)

之后根据需要重复此过程即可（或者直到出现 <eos> 这一结尾记号）。

这样一来，我们就可以生成新的文本。

这里需要注意的是，像上面这样生成的新文本是训练数据中没有的新生成的文本。

因为语言模型并不是背诵了训练数据，而是学习了训练数据中单词的排列模式。

如果语言模型通过语料库正确学习了单词的出现模式，我们就可以期待该语言模型生成的文本对人类而言是自然的、有意义的。

### 文本生成的实现

In [17]:
# ch07/rnnlm_gen.py
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm


class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        # 这个类用 generate(start_id, skip_ids, sample_size) 生成本文。
        # 此处，参数 start_id 是第 1 个单词 ID，sample_size 表示要采样的单词数量。
        # 另外，参数 skip_ids是单词 ID 列表（比如，[12, 20]），它指定的单词将不被采样。
        # 这个参数用于排除 PTB 数据集中的 <unk>、N 等被预处理过的单词。
        # PTB 数据集对原始文本进行了预处理，稀有单词被 <unk>替换，数字被 N替换。
        # 另外，我们用 <eos>作为文本的分隔符。
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x) # model.predict(x) 输出各个单词的得分 得分是正规化之前的值
            p = softmax(score.flatten()) # 归一化 得到概率

            sampled = np.random.choice(len(p), size=1, p=p) # 根据概率选择下一个单词
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)

现在，使用这个 RnnlmGen 类进行文本生成。这里先在完全没有学习的状态（即权重参数是随机初始值的状态）下生成文本

In [18]:
import numpy as np

In [19]:
# ch07/generate_text.py

# import sys
# sys.path.append('..')
import os
import pickle
# from rnnlm_gen import RnnlmGen
# from dataset import ptb


save_file = {
    'train':'ptb.train.npy',
    'test':'ptb.test.npy',
    'valid':'ptb.valid.npy'
}
vocab_file = 'ptb.vocab.pkl'

# dataset_dir = os.path.join(os.path.abspath('.'))
dataset_dir = os.path.join(os.path.abspath('..'), '神仔的代码', 'dataset')
dataset_dir

'D:\\06Jupyter\\03_code\\18深度学习进阶自然语言处理\\神仔的代码\\dataset'

In [20]:
def load_vocab():
    vocab_path = dataset_dir + '/' + vocab_file

    with open(vocab_path, 'rb') as f:
        word_to_id, id_to_word = pickle.load(f)
    return word_to_id, id_to_word


def load_data(data_type='train'):
    '''
        :param data_type: 数据的种类：'train' or 'test' or 'valid (val)'
        :return:
    '''
    if data_type == 'val': data_type = 'valid'
    save_path = os.path.join(dataset_dir, save_file[data_type])
    word_to_id, id_to_word = load_vocab()
    corpus = np.load(save_path)
    return corpus, word_to_id, id_to_word

In [21]:
corpus, word_to_id, id_to_word = load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

In [22]:
model = RnnlmGen()
# model.load_params('./dataset/Rnnlm.pkl')

In [23]:
# 设定start单词和skip单词
start_word = 'you' # 第 1 个单词是 you
start_id = word_to_id[start_word] # 将它的单词 ID 设为 start_id
skip_words = ['N', '<unk>', '$'] # 指定不参与采样的单词为 ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

In [24]:
# 文本生成
word_ids = model.generate(start_id, skip_ids)
word_ids # 生成文本的 generate() 方法返回单词 ID 列表

# 需要将单词 ID 列表转化为句子
txt = ' '.join([id_to_word[i] for i in word_ids]) 
# join() 方法通过“' 分隔符 '.join( 列表 )”这种形式连接单词。
txt = txt.replace(' <eos>', '.\n')
print(txt)

you mcnamee ultimately till asian inning been accustomed establishment vincent attract issuers seal floors stoll we dismal selling femina tissue leaders recognize manufactures revolution n.v. staffers summoned telling pons edged noise dubbed creatures styles claims pot meet was pot extra requiring individual indonesia factors benchmark appealed jobless policy respond airing climb chivas face-to-face cafeteria averaging roy hdtv granted war rebel madison examiner hard-line ed lipton longer germans gum cost-of-living norwegian buildup photograph saving chairman flow delaying sight creditors cadillac diamonds foreign-exchange departures businessmen dealerships lost direction disposal unpaid optimistic plead staffs quarter alabama stations budget encouraged ark mortgage-backed optimism advises


如你所见，输出的文本是一堆乱七八糟的单词。

不过这可以理解，因为这里的模型权重使用的是随机初始值，所以输出了没有意义的文本。

那么，如果换成学习好的语言模型，结果会怎样呢？我们利用上一章学习好的权重来进行文本生成。

In [25]:
corpus, word_to_id, id_to_word = load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

In [26]:
model = RnnlmGen()
model.load_params('./dataset/Rnnlm.pkl')

In [27]:
# 设定start单词和skip单词
start_word = 'you' # 第 1 个单词是 you
start_id = word_to_id[start_word] # 将它的单词 ID 设为 start_id
skip_words = ['N', '<unk>', '$'] # 指定不参与采样的单词为 ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

In [28]:
# 文本生成
word_ids = model.generate(start_id, skip_ids)
word_ids # 生成文本的 generate() 方法返回单词 ID 列表

# 需要将单词 ID 列表转化为句子
txt = ' '.join([id_to_word[i] for i in word_ids]) 
# join() 方法通过“' 分隔符 '.join( 列表 )”这种形式连接单词。
txt = txt.replace(' <eos>', '.\n')
print(txt)

you 'll be seen as weekend of the company.
 mr. fleischmann made a firm all linked off the proposals sharply and former president and senate of the chairman and chief executive officer thomas lawson.
 the board is investment in his daughters.
 we need political demands.
 donald e. evans the chairman of the firm 's pilots in new products is not one of his role in an money.
 we need to see that direct competition in multiples.
 its most aggressive junk banks received on its own consent development yesterday morning he believes that the chairman


```
you continually committed to general positions.
 again the figure could hit the world 's largest of brokerage firm d.c. as an analyst with poland.
 but in any chicago monetary fund the currency announced finance investors was morgan stanley & co. and credit markets which also remained small bid for a health offering and the merger in a dean basis.
 in addition a direct drug network ends mr. lawson 's merger while it 's not driven pressure to which further and expects an undisclosed amount for health.
 although both merrill lynch & co. was named president of
 ```

```
你一直致力于一般的职位。
作为波兰的分析师，这个数字可能再次冲击到世界上最大的经纪公司华盛顿特区。
但在芝加哥货币基金组织(chicago monetary fund)中，摩根士丹利(morgan stanley & co.)和信贷市场(credit markets)都是宣布的金融投资者。摩根士丹利和信贷市场也仍在以dean的名义小规模竞购医疗产品和合并。
此外，一个直接的毒品网络终止了劳森先生的合并，虽然它没有进一步驱动压力，并预期健康金额未披露。
尽管美林公司(merrill lynch & co.)都被任命为

```


虽然上面的结果中可以看到多处语法错误和意思不通的地方，不过
也有几处读起来已经比较像句子了。

### 更好的文本生成

如果有更好的语言模型，就可能有更好的文本。在上一章中，我们改进
了简单的 RNNLM，实现了“更好的 RNNLM”，将模型的困惑度从 136 降 至 75。现在，我们看一下这个“更好的 RNNLM”生成文本的能力。

In [29]:
class BetterRnnlmGen(BetterRnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x).flatten()
            p = softmax(score).flatten()

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        states = []
        for layer in self.lstm_layers:
            states.append((layer.h, layer.c))
        return states

    def set_state(self, states):
        for layer, state in zip(self.lstm_layers, states):
            layer.set_state(*state) 

In [30]:
# ch07/generate_better_text.py

# # coding: utf-8
# import sys
# sys.path.append('..')
# from common.np import *
# from rnnlm_gen import BetterRnnlmGen
# from dataset import ptb


corpus, word_to_id, id_to_word = load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

In [31]:
model = BetterRnnlmGen()
model.load_params('./dataset/BetterRnnlm.pkl')

In [32]:
# 设定start字符和skip字符
start_word = 'you'
start_id = word_to_id[start_word]

skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]
# 文本生成
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')

In [33]:
print(txt)

you can see whatever a lesser positive bank structured to get gold.
 the richmond va. real estate maker forecast a maximum life of the loss.
 the company said the market 's decision anytime soon but excellent earnings on friday 's sale did n't benefit any charges.
 for all of years plastics co. a diversified property construction and reinsurance concern were requests to make any divisions on rubble for hospitals in productivity.
 it said it near losses aimed at small beat the market which 's much tougher to engage in a sign as being negotiated by a


虽然这里生成的文本仍然存在若干问题（特别是语义方面），但是从某种程度上来说，这个更好的语言模型生成了更加自然的文本。

通过进一步改进这个模型，使用更大规模的语料库，应该能创造出更加自然的文本。

最后，我们尝试给这个更好的语言模型输入“the meaning of life is”，
让它生成后续的单词。为

了做这个实验，我们按顺序向模型输入 ['the', 'meaning', 'of', 'life']，进行正向传播。

此时不使用任何输出的结果，只是让 LSTM 层记住这些单词的信息。

然后，以单词 is 作为开始位置，生成“the meaning of life is”的后续内容。

In [34]:
model = BetterRnnlmGen()
model.load_params('./dataset/BetterRnnlm.pkl')
model.reset_state()

start_words = 'the meaning of life is'
start_ids = [word_to_id[w] for w in start_words.split(' ')]

for x in start_ids[:-1]:
    x = np.array(x).reshape(1, 1)
    model.predict(x)

In [35]:
word_ids = model.generate(start_ids[-1], skip_ids)
word_ids = start_ids[:-1] + word_ids
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print('-' * 50)
print(txt)

--------------------------------------------------
the meaning of life is the most recent busiest thing.
 since the revived article in her book maintained mayoral boards.
 it did n't have much hugo in my mind and this ought to work a policy and walk off it 's the best time.
 she says that if the attorney 's arrest first had seen the suit.
 charles a food firm based in irvine calif. was chartered by fossett inc. in a transaction valued at about three million shares.
 shearson lehman hutton inc. that split technological books indianapolis may begin a series of practice proposals combines the federal regulations


## seq2seq模型

### 针对的问题

时序数据-->时序数据

时序数据：文本数据、音频数据和视频数据都是时序数据。

应用场景：

- 机器翻译

- 语音识别等

- 聊天机器人应用

- 将源代码转为机器语言的编译器等

### 原理

seq2seq ==》 Encoder-Decoder 模型

![](./img/7-5.png)

Encoder（编码器）vs Decoder（解码器）

举例：

“A” ==》“1000001” ==》“A”

“吾輩は猫である”==》浓缩了翻译所必要信息的编码 ==》“I am a cat”

wagaha ihane kodearu

百香果 ==》 百香果冻干 ==》 蜜雪冰城满杯百香果

### 编码器

编码就是将任意长度的文本转换为一个固定长度的向量。

就好比任意大小的百香果，都变成一样大小的冻干方砖。

比如我们这种情况，编码器利用 RNN 将时序数据转换为隐藏状态 h。

![](./img/7-6.png)

说明：

1. 这里的RNN使用的是LSTM，可以使用“简单RNN”或者GRU等。

2. 输入要做一个分词

3. LSTM 的隐藏状态 h 是一个固定长度的向量

4. 最终输出 h 是 LSTM 层的最后一个隐藏状态，其中编码了翻译输入文本所需的信息


所以整体来讲就是 **任意长度的文本** 转换为 **固定长度的向量**


![](./img/7-7.png)

### 解码器

类似开篇说的文本生成器

解码器不是传了个h出来吗，这边接着

![](./img/7-8.png)

说明

1. `<eos>`  指示解码器的“开始 / 结束”

2. 在其他文献中，也有使用` <go>`、`<start>`或者`_`作为分隔符的例子。

### seq2seq整体

![](./img/7-9.png)

合在一起就是这样

### 时序数据处理案例：三位数加法学习

问题：三位数以内加法，看看seq2seq能否学习到加法运算的规则？

![](./img/7-10.png)


分词：在以字符为单位进行分割的，“57 + 5” ==》 ['5', '7', '+', '5']

输入长度：可变

“57 + 5” 4 个字符

“628 + 521” 7 个字符

批处理需要保证一个批次内各个样本的数据形状是一致的

解决方法：padding

在多余位置插入无效字符（这里是空白字符），从而使所有输入数据的长度对齐

![](./img/7-11.png)

本次的问题处理的是 0 ~ 999 的两个数的加法。

“999 + 999 = 1998”

输入的最大字符数是 7。

输出最大是 4 个字符

输出的开始处加上了分隔符“_”（下划线），使得输出数据的字符数统一为 5。

这个分隔符作为通知解码器开始生成文本的信号使用。

### 加法数据集

这里介绍的加法的学习数据预先存放在了 dataset/addition.txt 中。

dataset/sequence.py

专门处理  seq2seq 的学习数据

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import os
import numpy
# load_data() 和 get_vocab() 两个方法

id_to_char = {}
char_to_id = {}


def _update_vocab(txt):
    chars = list(txt)

    for i, char in enumerate(chars):
        if char not in char_to_id:
            tmp_id = len(char_to_id)
            char_to_id[char] = tmp_id
            id_to_char[tmp_id] = char


def load_data(file_name='addition.txt', seed=1984):
    # 读入由 file_name 指定的文本文件
    # 并将文本转换为字符 ID，返回训练数据和测试数据。

    # 该方法内部设有随机数种子seed 以打乱数据，分割训练数据和测试数据
    
    file_path = os.path.dirname(os.path.abspath(__file__)) + '/' + file_name

    if not os.path.exists(file_path):
        print('No file: %s' % file_name)
        return None

    questions, answers = [], []

    for line in open(file_path, 'r'):
        idx = line.find('_')
        questions.append(line[:idx])
        answers.append(line[idx:-1])

    # create vocab dict
    for i in range(len(questions)):
        q, a = questions[i], answers[i]
        _update_vocab(q)
        _update_vocab(a)

    # create numpy array
    x = numpy.zeros((len(questions), len(questions[0])), dtype=numpy.int)
    t = numpy.zeros((len(questions), len(answers[0])), dtype=numpy.int)

    for i, sentence in enumerate(questions):
        x[i] = [char_to_id[c] for c in list(sentence)]
    for i, sentence in enumerate(answers):
        t[i] = [char_to_id[c] for c in list(sentence)]

    # shuffle
    indices = numpy.arange(len(x))
    if seed is not None:
        numpy.random.seed(seed)
    numpy.random.shuffle(indices)
    x = x[indices]
    t = t[indices]

    # 10% for validation set
    split_at = len(x) - len(x) // 10
    (x_train, x_test) = x[:split_at], x[split_at:]
    (t_train, t_test) = t[:split_at], t[split_at:]

    return (x_train, t_train), (x_test, t_test)


def get_vocab():
    # get_vocab() 方法返回字符与 ID 的映射字典
    return char_to_id, id_to_char


In [19]:
# ch07/show_addition_dataset.py
# import sys
# sys.path.append('..')
from dataset import sequence # 使用 sequence 模块，可以轻松地读入 seq2seq 用的数据

# 简单起见，这里只分成训练数据和测试数据 2 份，用它们进行模型的训练和评价
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed=1984)

x_train 和 t_train 存放的是字符 ID

50 000 个加法样本

In [20]:
print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)


字符 ID 和字符之间的映射可以使用 char_to_id 和 id_to_char

In [21]:
char_to_id, id_to_char = sequence.get_vocab()

In [22]:
id_to_char

{0: '1',
 1: '6',
 2: '+',
 3: '7',
 4: '5',
 5: ' ',
 6: '_',
 7: '9',
 8: '2',
 9: '0',
 10: '3',
 11: '8',
 12: '4'}

In [11]:
char_to_id

{'1': 0,
 '6': 1,
 '+': 2,
 '7': 3,
 '5': 4,
 ' ': 5,
 '_': 6,
 '9': 7,
 '2': 8,
 '0': 9,
 '3': 10,
 '8': 11,
 '4': 12}

In [23]:
print(x_train[0])
print(t_train[0]) # target

[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]


In [24]:
print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

71+118 
_189 


数据集原本应分成训练用、验证用和测试用 3 份。

用训练数据进行学习，用验证数据进行调参，最后再用测试数据评价模型的能力。

## seq2seq的实现

seq2seq 是组合了两个 RNN 的神经网络。

这里我们首先将这两个 RNN=实现为 Encoder 类和 Decoder 类，然后将这两个类组合起来，来实现 seq2seq类。

我们先从 Encoder 类开始介绍。

### Encoder类

如图 7-13 所示，Encoder 类接收字符串，将其转化为向量 h。

![](./img/7-13.png)

如前所述，我们使用 RNN 实现编码器。这里，使用 LSTM 层实现图7-14 的层结构。

![](./img/7-14.png)

![](./img/7-15.png)

In [25]:
# ch07/seq2seq.py

# coding: utf-8
import sys
sys.path.append('..')
from common.time_layers import *
from common.base_model import BaseModel


class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # 初始化方法接收 vocab_size、wordvec_size 和 hidden_size 这 3 个参数。
        # vocab_size 是词汇量，相当于字符的种类。
        # 我们这个例子共有 13 种字符（数字 0 ~ 9、“+”、“ ”( 空白字符 )、“_”）。
        # wordvec_size 对应于字符向量的维数
        # hidden_size 对应于 LSTM 层的隐藏状态的维数。

        # 进行权重参数的初始化和层的生成。
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')

        self.embed = TimeEmbedding(embed_W)
        # 因为这次并不保持Time LSTM 层的状态，所以设定 stateful=False
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False)
        
        # 将权重参数和梯度分别归纳在成员变量 params 和 grads 的列表中
        self.params = self.embed.params + self.lstm.params
        self.grads = self.embed.grads + self.lstm.grads
        self.hs = None
        
        # 第 5 章和第 6 章的语言模型处理的是只有一个长时序数据的问题。
        # 那时我们设定 Time LSTM 层的参数 stateful=True，以在保持隐藏状态的同时，处理长时序数据。
        # 而这次是有多个短时序数据的问题。
        # 因此，针对每个问题重设 LSTM 的隐藏状态（为 0 向量）。

    def forward(self, xs):
        # 编码器的正向传播调用 Time Embedding 层 和 Time LSTM 层 的forward() 方法
        xs = self.embed.forward(xs)
        # 然后取出 Time LSTM 层的最后一个时刻的隐藏状态
        hs = self.lstm.forward(xs)
        self.hs = hs
        
        # 将它作为编码器的 forward() 方法的输出。
        return hs[:, -1, :]

    def backward(self, dh):
        # 在编码器的反向传播中，LSTM 层的最后一个隐藏状态的梯度是 dh，这个 dh 是从解码器传来的梯度。
        # 在反向传播的实现中，先生成元素为 0 的张量 dhs，再将 dh 存放到这个 dhs 中的对应位置。
        dhs = np.zeros_like(self.hs)
        dhs[:, -1, :] = dh
        
        # 剩下的就是调用 Time Embedding 层和 Time LSTM 层的 backward() 方法。
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout

### Decoder类

![](./img/7-16.png)

比如监督数据 _62 

此时输入数据是 ['_', '6', '2', ' ']，对应的输出是 ['6', '2', ' ', ' ']。


学习时和生成时的数据输入方法不同。

**学习时**

因为已经知道正确解，所以可以整体地输入时序方向上的数据。

使用softmax

![](./img/7-17.png)

**预测时**

（生成新字符串时），则只能输入第 1 个通知开始的分隔符（本次为“_”）。

然后，基于输出采样 1 个字符，并将这个采样出来的字符作为下一个输入，如此重复该过程。

![](./img/7-18.png)

我们这里不输出概率值，直接获得这是获取最大值的索引。

没有使用 Softmax 层，而是使用 argmax 从 Affine 层输出的得分中选择了最大值的字符 ID。

顺便说一句，在 7.1 节，在进行文本生成时，我们基于 Softmax 函数的概率分布进行了采样，因此生成的文本会随机变动。

因为这次的问题是加法，所以我们想消除这种概率性的“波动”，生成“确定性的”答案。

为此，这次我们仅选择得分最高的字符。也就是说，是“确定性”地选择，而不是“概率性”地选择。

图 7-18 显示了解码器生成字符串的过程

**整体**

在学习时和在生成时处理 Softmax 层的方式是不一样的。

因此，Decoder 类仅承担 Time Softmax with Loss 层之前的部分。

Softmax with Loss 层交给此后实现的 Seq2seq 类处理。

![](./img/7-19.png)

Decoder 类由 Time Embedding、Time LSTM 和Time Affine 这 3 个层构成。

Softmax 层对输入的向量进行正规化。

此时，向量元素的值虽然被改变，但是它们的大小关系没有变化。

因此，在图 7-18 的情况下，可以省略 Softmax 层。

In [16]:
class Decoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
       
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.embed = TimeEmbedding(embed_W)
        
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)
        
        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, h):
        self.lstm.set_state(h)

        out = self.embed.forward(xs)
        out = self.lstm.forward(out)
        score = self.affine.forward(out)
        return score

    # 学习时使用反向传播
    def backward(self, dscore): # 从上方的Softmax with Loss层接受梯度dscore
        
        # 按顺序反向传播
        dout = self.affine.backward(dscore)
        dout = self.lstm.backward(dout)
        dout = self.embed.backward(dout)
        dh = self.lstm.dh # Time LSTM层将时间方向的梯度保存在dh中 ，因此取出dh 作为return
        return dh
    
    # 文本生成时
    def generate(self, h, start_id, sample_size): 
        # 编码器接收隐藏状态h，最开始的输入字符id，生成的字符数量sample_size
        sampled = []
        sample_id = start_id
        self.lstm.set_state(h) # # 传入lstm层

        for _ in range(sample_size):
            # 输入一个字符
            x = np.array(sample_id).reshape((1, 1))
            # 正向计算
            out = self.embed.forward(x)
            out = self.lstm.forward(out)
            score = self.affine.forward(out)
            # 选择score最大的字符id
            sample_id = np.argmax(score.flatten())
            # 最为输出
            sampled.append(int(sample_id))

        return sampled

### Seq2seq类

只需要将Encoder类与Decoder类连接在一起

使用Time Softmax with Loss层计算损失

In [17]:
class Seq2seq(BaseModel):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = Decoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()

        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads

    def forward(self, xs, ts):
        decoder_xs, decoder_ts = ts[:, :-1], ts[:, 1:]

        h = self.encoder.forward(xs)
        score = self.decoder.forward(decoder_xs, h)
        loss = self.softmax.forward(score, decoder_ts)
        return loss

    def backward(self, dout=1):
        dout = self.softmax.backward(dout)
        dh = self.decoder.backward(dout)
        dout = self.encoder.backward(dh)
        return dout

    def generate(self, xs, start_id, sample_size):
        h = self.encoder.forward(xs)
        sampled = self.decoder.generate(h, start_id, sample_size)
        return sampled

### 加法问题训练与评价

学习流程：

1. 从训练数据中选择一个 mini-batch

2. 基于 mini-batch 计算梯度

3. 使用梯度更新权重

评估指标：seq2seq 针对每个 epoch 求解测试数据（生成字符串），并计算正确率。

seq2seq 的学习代码如下所示

In [18]:
# ch07/train_seq2seq.py

# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
# from seq2seq import Seq2seq
# from peeky_seq2seq import PeekySeq2seq


# 读入数据集
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()


# 设定超参数
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0


model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc * 100))

# 绘制图形
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.show()

| epoch 1 |  iter 1 / 351 | time 0[s] | loss 2.56
| epoch 1 |  iter 21 / 351 | time 1[s] | loss 2.53
| epoch 1 |  iter 41 / 351 | time 2[s] | loss 2.17
| epoch 1 |  iter 61 / 351 | time 3[s] | loss 1.96
| epoch 1 |  iter 81 / 351 | time 4[s] | loss 1.92
| epoch 1 |  iter 101 / 351 | time 5[s] | loss 1.87
| epoch 1 |  iter 121 / 351 | time 6[s] | loss 1.85
| epoch 1 |  iter 141 / 351 | time 7[s] | loss 1.83
| epoch 1 |  iter 161 / 351 | time 8[s] | loss 1.79
| epoch 1 |  iter 181 / 351 | time 10[s] | loss 1.77
| epoch 1 |  iter 201 / 351 | time 11[s] | loss 1.77
| epoch 1 |  iter 221 / 351 | time 12[s] | loss 1.76
| epoch 1 |  iter 241 / 351 | time 13[s] | loss 1.76
| epoch 1 |  iter 261 / 351 | time 14[s] | loss 1.76
| epoch 1 |  iter 281 / 351 | time 15[s] | loss 1.75
| epoch 1 |  iter 301 / 351 | time 16[s] | loss 1.74
| epoch 1 |  iter 321 / 351 | time 18[s] | loss 1.75
| epoch 1 |  iter 341 / 351 | time 19[s] | loss 1.74
Q 77+85  
T 162 
X 100 
---
Q 975+164
T 1139
X 1000
---
Q 582

| epoch 7 |  iter 321 / 351 | time 20[s] | loss 1.06
| epoch 7 |  iter 341 / 351 | time 21[s] | loss 1.04
Q 77+85  
T 162 
X 156 
---
Q 975+164
T 1139
X 1160
---
Q 582+84 
T 666 
X 656 
---
Q 8+155  
T 163 
X 141 
---
Q 367+55 
T 422 
X 402 
---
Q 600+257
T 857 
X 856 
---
Q 761+292
T 1053
X 1031
---
Q 830+597
T 1427
X 1374
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
X 217 
---
val acc 3.180%
| epoch 8 |  iter 1 / 351 | time 0[s] | loss 1.06
| epoch 8 |  iter 21 / 351 | time 1[s] | loss 1.06
| epoch 8 |  iter 41 / 351 | time 2[s] | loss 1.08
| epoch 8 |  iter 61 / 351 | time 4[s] | loss 1.04
| epoch 8 |  iter 81 / 351 | time 5[s] | loss 1.08
| epoch 8 |  iter 101 / 351 | time 6[s] | loss 1.06
| epoch 8 |  iter 121 / 351 | time 8[s] | loss 1.04
| epoch 8 |  iter 141 / 351 | time 9[s] | loss 1.04
| epoch 8 |  iter 161 / 351 | time 10[s] | loss 1.04
| epoch 8 |  iter 181 / 351 | time 12[s] | loss 1.03
| epoch 8 |  iter 201 / 351 | time 13[s] | loss 1.03
| epoch 8 |  iter 221 / 351 | 

| epoch 14 |  iter 121 / 351 | time 7[s] | loss 0.91
| epoch 14 |  iter 141 / 351 | time 8[s] | loss 0.91
| epoch 14 |  iter 161 / 351 | time 10[s] | loss 0.91
| epoch 14 |  iter 181 / 351 | time 11[s] | loss 0.90
| epoch 14 |  iter 201 / 351 | time 12[s] | loss 0.87
| epoch 14 |  iter 221 / 351 | time 13[s] | loss 0.90
| epoch 14 |  iter 241 / 351 | time 15[s] | loss 0.92
| epoch 14 |  iter 261 / 351 | time 16[s] | loss 0.92
| epoch 14 |  iter 281 / 351 | time 17[s] | loss 0.90
| epoch 14 |  iter 301 / 351 | time 18[s] | loss 0.89
| epoch 14 |  iter 321 / 351 | time 20[s] | loss 0.89
| epoch 14 |  iter 341 / 351 | time 21[s] | loss 0.90
Q 77+85  
T 162 
X 161 
---
Q 975+164
T 1139
X 1121
---
Q 582+84 
T 666 
X 658 
---
Q 8+155  
T 163 
X 171 
---
Q 367+55 
T 422 
X 419 
---
Q 600+257
T 857 
X 859 
---
Q 761+292
T 1053
X 1070
---
Q 830+597
T 1427
X 1418
---
Q 26+838 
T 864 
X 859 
---
Q 143+93 
T 236 
X 238 
---
val acc 7.640%
| epoch 15 |  iter 1 / 351 | time 0[s] | loss 0.86
| epoch 

val acc 4.800%
| epoch 21 |  iter 1 / 351 | time 0[s] | loss 0.88
| epoch 21 |  iter 21 / 351 | time 1[s] | loss 0.83
| epoch 21 |  iter 41 / 351 | time 2[s] | loss 0.83
| epoch 21 |  iter 61 / 351 | time 3[s] | loss 0.81
| epoch 21 |  iter 81 / 351 | time 5[s] | loss 0.84
| epoch 21 |  iter 101 / 351 | time 6[s] | loss 0.81
| epoch 21 |  iter 121 / 351 | time 7[s] | loss 0.82
| epoch 21 |  iter 141 / 351 | time 8[s] | loss 0.88
| epoch 21 |  iter 161 / 351 | time 10[s] | loss 0.81
| epoch 21 |  iter 181 / 351 | time 11[s] | loss 0.80
| epoch 21 |  iter 201 / 351 | time 12[s] | loss 0.79
| epoch 21 |  iter 221 / 351 | time 13[s] | loss 0.80
| epoch 21 |  iter 241 / 351 | time 15[s] | loss 0.80
| epoch 21 |  iter 261 / 351 | time 16[s] | loss 0.80
| epoch 21 |  iter 281 / 351 | time 17[s] | loss 0.84
| epoch 21 |  iter 301 / 351 | time 19[s] | loss 0.82
| epoch 21 |  iter 321 / 351 | time 20[s] | loss 0.83
| epoch 21 |  iter 341 / 351 | time 21[s] | loss 0.82
Q 77+85  
T 162 
O 162 
---

精确度计算

In [ ]:
# common/util.py 

# 这个方法向模型输入问题
# 生成字符串，并判断它是否与答案相符
# 如果模型给出的答案正确，则返回 1；如果错误，则返回 0
def eval_seq2seq(model, question, correct, id_to_char,
                 verbos=False, is_reverse=False):
    # 模型 model
    # 问题（字符 ID 数组）question
    # 正确解（字符 ID 列表）correct。
    # 字符 ID与字符映射的字典 id_to_char
    # 指定是否显示结果的 verbose
    # 指定是否反转输入语句的 is_reverse 关于参数 is_reverse，我们稍后再解释
    # 如果设置 verbose = True，则结果会显示在终端上。这次的实验仅显示最初的 10 份测试数据。

    correct = correct.flatten()
    # 开头的分隔符
    start_id = correct[0]
    correct = correct[1:]
    guess = model.generate(question, start_id, len(correct))

    # 转换为字符串
    question = ''.join([id_to_char[int(c)] for c in question.flatten()])
    correct = ''.join([id_to_char[int(c)] for c in correct])
    guess = ''.join([id_to_char[int(c)] for c in guess])

    if verbos:
        if is_reverse:
            question = question[::-1]

        colors = {'ok': '\033[92m', 'fail': '\033[91m', 'close': '\033[0m'}
        print('Q', question)
        print('T', correct)

        is_windows = os.name == 'nt'

        if correct == guess:
            mark = colors['ok'] + '☑' + colors['close']
            if is_windows:
                mark = 'O'
            print(mark + ' ' + guess)
        else:
            mark = colors['fail'] + '☒' + colors['close']
            if is_windows:
                mark = 'X'
            print(mark + ' ' + guess)
        print('---')

    return 1 if guess == correct else 0

随着学习的积累，正确率稳步提高。

本次的实验只进行了 25 次，最后的正确率约为 10%。

从图中的变化趋势可知，如果继续学习，正确率应该还会进一步上升。

不过，为了能更好地学习相同的问题（加法问题），这里我们暂停本次学习，对 seq2seq 进行一些改进。

## seq2seq的改进

### 反转输入数据 Revers

这个反转输入数据的技巧是文献 [41] 中提出来的。

据研究，在许多情况下，使用这个技巧后，学习进展得更快，最终的精度也有提高。

现在我们来做一下实验。

在读入数据集之后，我们追加下面的代码

![](./img/7-23.png)

In [28]:
# ch07/train_seq2seq.py

# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
# from seq2seq import Seq2seq
# from peeky_seq2seq import PeekySeq2seq


# 读入数据集
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

In [29]:
x_train[0]

array([ 3,  0,  2,  0,  0, 11,  5])

In [30]:
# Reverse input? =================================================
is_reverse = True  # True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

In [31]:
x_train[0]

array([ 5, 11,  0,  0,  2,  0,  3])

In [32]:
# 设定超参数
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc * 100))

# 绘制图形
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.show()

| epoch 1 |  iter 1 / 351 | time 0[s] | loss 2.56
| epoch 1 |  iter 21 / 351 | time 1[s] | loss 2.52
| epoch 1 |  iter 41 / 351 | time 2[s] | loss 2.17
| epoch 1 |  iter 61 / 351 | time 3[s] | loss 1.96
| epoch 1 |  iter 81 / 351 | time 4[s] | loss 1.91
| epoch 1 |  iter 101 / 351 | time 5[s] | loss 1.87
| epoch 1 |  iter 121 / 351 | time 7[s] | loss 1.86
| epoch 1 |  iter 141 / 351 | time 8[s] | loss 1.84
| epoch 1 |  iter 161 / 351 | time 10[s] | loss 1.80
| epoch 1 |  iter 181 / 351 | time 12[s] | loss 1.78
| epoch 1 |  iter 201 / 351 | time 13[s] | loss 1.77
| epoch 1 |  iter 221 / 351 | time 15[s] | loss 1.77
| epoch 1 |  iter 241 / 351 | time 16[s] | loss 1.76
| epoch 1 |  iter 261 / 351 | time 18[s] | loss 1.75
| epoch 1 |  iter 281 / 351 | time 19[s] | loss 1.74
| epoch 1 |  iter 301 / 351 | time 21[s] | loss 1.74
| epoch 1 |  iter 321 / 351 | time 22[s] | loss 1.74
| epoch 1 |  iter 341 / 351 | time 24[s] | loss 1.73
Q 77+85  
T 162 
X 100 
---
Q 975+164
T 1139
X 1000
---
Q 58

| epoch 7 |  iter 301 / 351 | time 20[s] | loss 0.63
| epoch 7 |  iter 321 / 351 | time 21[s] | loss 0.63
| epoch 7 |  iter 341 / 351 | time 22[s] | loss 0.62
Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
X 1142
---
Q 582+84 
T 666 
O 666 
---
Q 8+155  
T 163 
X 162 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
X 859 
---
Q 761+292
T 1053
X 1144
---
Q 830+597
T 1427
X 1431
---
Q 26+838 
T 864 
X 866 
---
Q 143+93 
T 236 
X 239 
---
val acc 17.500%
| epoch 8 |  iter 1 / 351 | time 0[s] | loss 0.66
| epoch 8 |  iter 21 / 351 | time 1[s] | loss 0.61
| epoch 8 |  iter 41 / 351 | time 2[s] | loss 0.62
| epoch 8 |  iter 61 / 351 | time 4[s] | loss 0.61
| epoch 8 |  iter 81 / 351 | time 5[s] | loss 0.61
| epoch 8 |  iter 101 / 351 | time 6[s] | loss 0.61
| epoch 8 |  iter 121 / 351 | time 8[s] | loss 0.60
| epoch 8 |  iter 141 / 351 | time 9[s] | loss 0.60
| epoch 8 |  iter 161 / 351 | time 10[s] | loss 0.59
| epoch 8 |  iter 181 / 351 | time 12[s] | loss 0.58
| epoch 8 |  iter 201 / 351 |

| epoch 14 |  iter 101 / 351 | time 7[s] | loss 0.41
| epoch 14 |  iter 121 / 351 | time 8[s] | loss 0.39
| epoch 14 |  iter 141 / 351 | time 9[s] | loss 0.39
| epoch 14 |  iter 161 / 351 | time 11[s] | loss 0.38
| epoch 14 |  iter 181 / 351 | time 12[s] | loss 0.38
| epoch 14 |  iter 201 / 351 | time 14[s] | loss 0.38
| epoch 14 |  iter 221 / 351 | time 15[s] | loss 0.38
| epoch 14 |  iter 241 / 351 | time 16[s] | loss 0.39
| epoch 14 |  iter 261 / 351 | time 18[s] | loss 0.40
| epoch 14 |  iter 281 / 351 | time 19[s] | loss 0.41
| epoch 14 |  iter 301 / 351 | time 21[s] | loss 0.39
| epoch 14 |  iter 321 / 351 | time 22[s] | loss 0.39
| epoch 14 |  iter 341 / 351 | time 23[s] | loss 0.39
Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
X 1137
---
Q 582+84 
T 666 
X 667 
---
Q 8+155  
T 163 
O 163 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
X 858 
---
Q 761+292
T 1053
O 1053
---
Q 830+597
T 1427
X 1426
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
X 235 
---
val acc 36.640%
| epo

Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
O 1139
---
Q 582+84 
T 666 
X 665 
---
Q 8+155  
T 163 
X 162 
---
Q 367+55 
T 422 
X 421 
---
Q 600+257
T 857 
X 859 
---
Q 761+292
T 1053
X 1051
---
Q 830+597
T 1427
X 1428
---
Q 26+838 
T 864 
X 862 
---
Q 143+93 
T 236 
O 236 
---
val acc 50.140%
| epoch 21 |  iter 1 / 351 | time 0[s] | loss 0.31
| epoch 21 |  iter 21 / 351 | time 1[s] | loss 0.31
| epoch 21 |  iter 41 / 351 | time 2[s] | loss 0.30
| epoch 21 |  iter 61 / 351 | time 4[s] | loss 0.30
| epoch 21 |  iter 81 / 351 | time 6[s] | loss 0.32
| epoch 21 |  iter 101 / 351 | time 8[s] | loss 0.32
| epoch 21 |  iter 121 / 351 | time 9[s] | loss 0.31
| epoch 21 |  iter 141 / 351 | time 10[s] | loss 0.30
| epoch 21 |  iter 161 / 351 | time 12[s] | loss 0.31
| epoch 21 |  iter 181 / 351 | time 13[s] | loss 0.31
| epoch 21 |  iter 201 / 351 | time 15[s] | loss 0.31
| epoch 21 |  iter 221 / 351 | time 16[s] | loss 0.32
| epoch 21 |  iter 241 / 351 | time 17[s] | loss 0.31
| epoch 21 |  i

注意，仅仅通过反转输入数据，没有增加数据啥的，学习的进展就得到了改善！

在 25 个 epoch 时，正确率为 50% 左右。

为什么反转数据后，学习进展变快，精度提高了呢？

虽然理论上不是很清楚，但是直观上可以认为，反转数据后梯度的传播可以更平滑。

比如，考虑将“吾輩 は 猫 で ある”A 翻译成“I am a cat”这一问题，单词“吾
輩”和单词“I”之间有转换关系。

此时，从“吾輩”到“I”的路程必须经过“は”“猫”“で”“ある”这 4 个单词的 LSTM 层。

因此，在反向传播时，梯度从“I”抵达“吾輩”，也要受到这个距离的影响

那么，如果反转输入语句，也就是变为“ある で 猫 は 吾輩”，结果会怎样呢？

此时，“吾輩”和“I”彼此相邻，梯度可以直接传递。

如此，因为通过反转，输入语句的开始部分和对应的转换后的单词之间的距离变近（这样的情况变多），所以梯度的传播变得更容易，学习效率也更高。

不过，在反转输入数据后，单词之间的“平均”距离并不会发生改变。

### 偷窥 Peeky

**BEFORE**

只有最开始时刻的 LSTM 层利用了 h

![](./img/7-25.png)

**AFTER**

将编码器的输出 h 分配给所有时刻的 Affine 层和 LSTM 层。

![](./img/7-26.png)

这里的改进是将编码好的信息分配给解码器的其他层，这可以解释为其他层也能“偷窥”到编码信息。

因为“偷窥”的英语是 peek，所以将这个改进了的解码器称为 Peeky Decoder。

同理，将使用了Peeky Decoder 的 seq2seq 称 为 Peeky seq2seq。

这个想法基于文献 [42]

**改动**

有两个向量同时被输入到了 LSTM 层和 Affine 层，这实际上表示两个向量的拼接（concatenate）。

因此，在刚才的图中，如果使用concat 节点拼接两个向量

![](./img/7-27.png)

只需要改Decoder

In [34]:
# ch07/peeky_seq2seq.py

# coding: utf-8
import sys
sys.path.append('..')
from common.time_layers import *
# from seq2seq import Seq2seq, Encoder


class PeekyDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # PeekyDecoder 的初始化和上一节的 Decoder 基本上是一样的
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(H + D, 4 * H) / np.sqrt(H + D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(H + H, V) / np.sqrt(H + H)).astype('f') # 不同之处仅在于 LSTM 层权重和 Affine 层权重的形状
        affine_b = np.zeros(V).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.affine = TimeAffine(affine_W, affine_b)

        self.params, self.grads = [], []
        for layer in (self.embed, self.lstm, self.affine):
            self.params += layer.params
            self.grads += layer.grads
        self.cache = None

    def forward(self, xs, h):
        N, T = xs.shape
        N, H = h.shape

        self.lstm.set_state(h)

        out = self.embed.forward(xs)
        # 这里首先使用 np.repeat() 根据时序大小复制相应份数的 h，并将其设置为 hs。
        hs = np.repeat(h, T, axis=0).reshape(N, T, H) 
        # 然后，将 hs 和 Embedding 层的输出用np.concatenate() 拼接，并输入 LSTM 层
        out = np.concatenate((hs, out), axis=2)
        out = self.lstm.forward(out)
        # 同样地，Affine 层的输入也是 hs和 LSTM 层的输出的拼接。
        out = np.concatenate((hs, out), axis=2)

        score = self.affine.forward(out)
        self.cache = H
        return score

    def backward(self, dscore):
        H = self.cache

        dout = self.affine.backward(dscore)
        dout, dhs0 = dout[:, :, H:], dout[:, :, :H]
        dout = self.lstm.backward(dout)
        dembed, dhs1 = dout[:, :, H:], dout[:, :, :H]
        self.embed.backward(dembed)

        dhs = dhs0 + dhs1
        dh = self.lstm.dh + np.sum(dhs, axis=1)
        return dh

    def generate(self, h, start_id, sample_size):
        sampled = []
        char_id = start_id
        self.lstm.set_state(h)

        H = h.shape[1]
        peeky_h = h.reshape(1, 1, H)
        for _ in range(sample_size):
            x = np.array([char_id]).reshape((1, 1))
            out = self.embed.forward(x)

            out = np.concatenate((peeky_h, out), axis=2)
            out = self.lstm.forward(out)
            out = np.concatenate((peeky_h, out), axis=2)
            score = self.affine.forward(out)

            char_id = np.argmax(score.flatten())
            sampled.append(char_id)

        return sampled


class PeekySeq2seq(Seq2seq):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        # PeekySeq2seq和上一节的 Seq2seq 类基本相同
        # 唯一的区别是 Decoder 层。
        # Decoder 类 ==》PeekyDecoder，剩余的逻辑完全一样。
        # 因此，PeekySeq2seq类的实现只需要继承上一章的 Seq2seq 类，并修改一下初始化部分
        
        V, D, H = vocab_size, wordvec_size, hidden_size
        self.encoder = Encoder(V, D, H)
        self.decoder = PeekyDecoder(V, D, H)
        self.softmax = TimeSoftmaxWithLoss()

        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads


In [35]:
# ch07/train_seq2seq.py

# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
# from seq2seq import Seq2seq
# from peeky_seq2seq import PeekySeq2seq


# 读入数据集
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

# Reverse input? =================================================
is_reverse = True  # True
if is_reverse:
    x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
# ================================================================

# 设定超参数
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# Normal or Peeky? ==============================================
# model = Seq2seq(vocab_size, wordvec_size, hidden_size)
model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)
# ================================================================
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch):
    trainer.fit(x_train, t_train, max_epoch=1,
                batch_size=batch_size, max_grad=max_grad)

    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct,
                                    id_to_char, verbose, is_reverse)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc * 100))

# 绘制图形
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.show()

| epoch 1 |  iter 1 / 351 | time 0[s] | loss 2.57
| epoch 1 |  iter 21 / 351 | time 1[s] | loss 2.48
| epoch 1 |  iter 41 / 351 | time 2[s] | loss 2.20
| epoch 1 |  iter 61 / 351 | time 3[s] | loss 1.99
| epoch 1 |  iter 81 / 351 | time 5[s] | loss 1.89
| epoch 1 |  iter 101 / 351 | time 6[s] | loss 1.82
| epoch 1 |  iter 121 / 351 | time 8[s] | loss 1.82
| epoch 1 |  iter 141 / 351 | time 9[s] | loss 1.80
| epoch 1 |  iter 161 / 351 | time 11[s] | loss 1.79
| epoch 1 |  iter 181 / 351 | time 12[s] | loss 1.78
| epoch 1 |  iter 201 / 351 | time 14[s] | loss 1.77
| epoch 1 |  iter 221 / 351 | time 15[s] | loss 1.76
| epoch 1 |  iter 241 / 351 | time 16[s] | loss 1.76
| epoch 1 |  iter 261 / 351 | time 18[s] | loss 1.75
| epoch 1 |  iter 281 / 351 | time 19[s] | loss 1.74
| epoch 1 |  iter 301 / 351 | time 21[s] | loss 1.74
| epoch 1 |  iter 321 / 351 | time 22[s] | loss 1.73
| epoch 1 |  iter 341 / 351 | time 24[s] | loss 1.73
Q 77+85  
T 162 
X 100 
---
Q 975+164
T 1139
X 1013
---
Q 58

| epoch 7 |  iter 301 / 351 | time 25[s] | loss 0.55
| epoch 7 |  iter 321 / 351 | time 27[s] | loss 0.54
| epoch 7 |  iter 341 / 351 | time 28[s] | loss 0.53
Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
O 1139
---
Q 582+84 
T 666 
X 665 
---
Q 8+155  
T 163 
X 156 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
X 858 
---
Q 761+292
T 1053
X 1052
---
Q 830+597
T 1427
X 1428
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
X 235 
---
val acc 39.100%
| epoch 8 |  iter 1 / 351 | time 0[s] | loss 0.51
| epoch 8 |  iter 21 / 351 | time 4[s] | loss 0.50
| epoch 8 |  iter 41 / 351 | time 7[s] | loss 0.49
| epoch 8 |  iter 61 / 351 | time 8[s] | loss 0.48
| epoch 8 |  iter 81 / 351 | time 10[s] | loss 0.47
| epoch 8 |  iter 101 / 351 | time 12[s] | loss 0.46
| epoch 8 |  iter 121 / 351 | time 13[s] | loss 0.46
| epoch 8 |  iter 141 / 351 | time 14[s] | loss 0.44
| epoch 8 |  iter 161 / 351 | time 16[s] | loss 0.41
| epoch 8 |  iter 181 / 351 | time 17[s] | loss 0.42
| epoch 8 |  iter 201 / 3

| epoch 14 |  iter 101 / 351 | time 6[s] | loss 0.05
| epoch 14 |  iter 121 / 351 | time 7[s] | loss 0.05
| epoch 14 |  iter 141 / 351 | time 9[s] | loss 0.05
| epoch 14 |  iter 161 / 351 | time 10[s] | loss 0.05
| epoch 14 |  iter 181 / 351 | time 11[s] | loss 0.05
| epoch 14 |  iter 201 / 351 | time 13[s] | loss 0.05
| epoch 14 |  iter 221 / 351 | time 14[s] | loss 0.06
| epoch 14 |  iter 241 / 351 | time 15[s] | loss 0.06
| epoch 14 |  iter 261 / 351 | time 16[s] | loss 0.07
| epoch 14 |  iter 281 / 351 | time 18[s] | loss 0.06
| epoch 14 |  iter 301 / 351 | time 19[s] | loss 0.06
| epoch 14 |  iter 321 / 351 | time 20[s] | loss 0.05
| epoch 14 |  iter 341 / 351 | time 22[s] | loss 0.05
Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
O 1139
---
Q 582+84 
T 666 
O 666 
---
Q 8+155  
T 163 
O 163 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
O 857 
---
Q 761+292
T 1053
O 1053
---
Q 830+597
T 1427
O 1427
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
O 236 
---
val acc 94.660%
| epo

Q 77+85  
T 162 
O 162 
---
Q 975+164
T 1139
O 1139
---
Q 582+84 
T 666 
O 666 
---
Q 8+155  
T 163 
O 163 
---
Q 367+55 
T 422 
O 422 
---
Q 600+257
T 857 
O 857 
---
Q 761+292
T 1053
O 1053
---
Q 830+597
T 1427
O 1427
---
Q 26+838 
T 864 
O 864 
---
Q 143+93 
T 236 
O 236 
---
val acc 95.120%
| epoch 21 |  iter 1 / 351 | time 0[s] | loss 0.03
| epoch 21 |  iter 21 / 351 | time 1[s] | loss 0.02
| epoch 21 |  iter 41 / 351 | time 2[s] | loss 0.02
| epoch 21 |  iter 61 / 351 | time 4[s] | loss 0.02
| epoch 21 |  iter 81 / 351 | time 5[s] | loss 0.02
| epoch 21 |  iter 101 / 351 | time 6[s] | loss 0.02
| epoch 21 |  iter 121 / 351 | time 7[s] | loss 0.02
| epoch 21 |  iter 141 / 351 | time 9[s] | loss 0.02
| epoch 21 |  iter 161 / 351 | time 10[s] | loss 0.02
| epoch 21 |  iter 181 / 351 | time 11[s] | loss 0.01
| epoch 21 |  iter 201 / 351 | time 13[s] | loss 0.02
| epoch 21 |  iter 221 / 351 | time 14[s] | loss 0.01
| epoch 21 |  iter 241 / 351 | time 15[s] | loss 0.01
| epoch 21 |  it

加上了 Peeky 的 seq2seq 的结果大幅变好。

刚过 10 个epoch 时，正确率已经超过 90%，最终的正确率接近 100%。

从上述实验结果可知，Reverse 和 Peeky 都有很好的效果。

借助反转输入语句的 Reverse 和共享编码器信息的 Peeky，我们获得了令人满意的结果！

这样我们就结束了对 seq2seq 的改进，不过故事仍在继续。

实际上，本节的改进只能说是“小改进”，下一章我们将对 seq2seq 进行“大改进”。

届时将使用名为 Attention 的技术，它能使 seq2seq 发生巨大变化。

这里的实验有几个需要注意的地方。

因为使用 Peeky 后，网络的权重参数会额外地增加，计算量也会增加，所以这里的实验结果必须考虑到相应地增加的“负担”。

另外，seq2seq 的精度会随着超参数的调整而大幅变化。

虽然这里的结果是可靠的，但是在实际问题中，它的效果可能不稳定。

## seq2seq的应用

### 聊天机器人

![](./img/7-29.png)

### 算法学习

44

![](./img/7-30.png)

### 自动图像描述

![](./img/7-31.png)

![](./img/7-32.png)

的自动图像描述（image captioning）[45] [46]。

seq2seq 相关

[40] Implementation of sequence to sequence learning for performing 
addition of two numbers (as strings).

[41] Sutskever, Ilya, Oriol Vinyals, Quoc V. Le.Sequence to sequence 
learning with neural networks[J]. Advances in neural information 
processing systems, 2014.

[42] Cho, Kyunghyun, et al.Learning phrase representations using RNN 
encoder-decoder for statistical machine translation[J]. arXiv preprint 
arXiv:1406.1078, 2014.

[43] Vinyals, Oriol, Quoc Le.A neural conversational model[J]. arXiv 
preprint arXiv:1506.05869, 2015.

[44] Zaremba, Wojciech, Ilya Sutskever.Learning to execute[J]. arXiv 
preprint arXiv:1410.4615, 2014.

[45] Vinyals, Oriol, et al.Show and tell: A neural image caption 
generator[J].Computer Vision and Pattern Recognition (CVPR), 
2015 IEEE Conference on. IEEE, 2015.

[46] Karpathy, Andrej, Li Fei-Fei.Deep visual-semantic alignments for 
generating image descriptions[J]. Proceedings of the IEEE conference 
on computer vision and pattern recognition, 2015.

[47] Show and Tell: A Neural Image Caption Generator.

## 课后习题

观看视频并回答以下问题

https://www.bilibili.com/video/BV1fy4y187cf?from=search&seid=8401372596628816366

1. 温周分别对应了本章的哪个概念

2. 温念的诗对应了本章哪个概念

3. 周子舒为何会头疼